# <div align="center" style="color: #ff5733;">TSA Closure Third Notification</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

# Ask Balasubramani to refresh risk_credit_mis.tsa_account_closure table.
**Check if that is refreshed when Balasubramani confirms it**

In [2]:
sq = """select Run_Date, count(1)cnt from risk_credit_mis.tsa_account_closure group by 1 order by 1 desc"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
df.head(1)

Job ID 16d6328d-524d-4efe-8b16-0015c65a6b19 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,Run_Date,cnt
0,2024-05-23,555967


The date should show yesterday's date

# Read the data from the first sheet for Type1 and Type2 data provided by Marvin

In [3]:
d1 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\March24&April24 Ongoing Batch\ThirdNotification_Files\Type 1 & 2 - 2nd Notification April.xlsx")
d1.shape

(7542, 2)

In [4]:
print(f"The unique count of customer_id is:\t{len(d1['Customer_Id'].unique())}")
d1.head()

The unique count of customer_id is:	7542


,Customer_Id,ACTION_TYPE
0,2390807,TYPE 1
1,2407245,TYPE 1
2,2380295,TYPE 1
3,2392393,TYPE 1
4,2396066,TYPE 1


# Read the data from the second sheet for Type3 data provided by Marvin

In [5]:
d2 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\March24&April24 Ongoing Batch\ThirdNotification_Files\Type 3 - 2nd Notification April.xlsx")
d2.shape

(1062, 2)

In [6]:
print(f"The unique count of customer_id is:\t{len(d2['Customer_Id'].unique())}")
d2.head()

The unique count of customer_id is:	1062


,Customer_Id,ACTION_TYPE
0,1212254,TYPE 3
1,2226881,TYPE 3
2,2206744,TYPE 3
3,1896790,TYPE 3
4,1523340,TYPE 3


In [7]:
result = pd.concat([d1, d2], axis=0, ignore_index=True)
result.shape

(8604, 2)

In [8]:
result.ACTION_TYPE.value_counts()

ACTION_TYPE
TYPE 1    7522
TYPE 3    1062
TYPE 2      20
Name: count, dtype: int64

In [12]:
result[result['Customer_Id'] == 1896790]

,Customer_Id,ACTION_TYPE
7545,1896790,TYPE 3


In [13]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8604 entries, 0 to 8603
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  8604 non-null   int64 
 1   ACTION_TYPE  8604 non-null   object
dtypes: int64(1), object(1)
memory usage: 134.6+ KB


In [14]:
result['Customer_Id'] = result['Customer_Id'].astype(str)
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8604 entries, 0 to 8603
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  8604 non-null   object
 1   ACTION_TYPE  8604 non-null   object
dtypes: object(2)
memory usage: 134.6+ KB


In [15]:
result[result['Customer_Id'] == '1896790']

,Customer_Id,ACTION_TYPE
7545,1896790,TYPE 3


In [16]:
result.to_csv("temp.csv", index = False)

In [17]:
a = tuple(result['Customer_Id'])
a[:5]

('2390807', '2407245', '2380295', '2392393', '2396066')

In [18]:
d = '2024-05-23'   ### Need to change this to D-1 everytime we run this code ###
# Get the current date
current_date = pd.Timestamp.now().date()
d_minus_1_days = current_date - pd.DateOffset(days=1)
# Calculate D-60 days
d_minus_60_days = '2024-02-23'
# Calculate D-180 days
d_minus_180_days = '2023-10-26'

print("D-1 day from current date", d_minus_1_days )
print("D-60 day from the current date - 1:", d_minus_60_days)
print("D-180 day from the current date -1:", d_minus_180_days)

d_minus_1_days_string = d_minus_1_days.strftime('%Y-%m-%d')
print("D-1 day from the current date:", d_minus_1_days_string)

d_minus_60_days_string = d_minus_60_days
print("D-60 day from the current date:", d_minus_60_days_string)

d_minus_180_days_string = d_minus_180_days
print("D-180 day from the current date:", d_minus_180_days_string)

D-1 day from current date 2024-05-23 00:00:00
D-60 day from the current date - 1: 2024-02-23
D-180 day from the current date -1: 2023-10-26
D-1 day from the current date: 2024-05-23
D-60 day from the current date: 2024-02-23
D-180 day from the current date: 2023-10-26


The date should be D-1? If the date is not D-1 then ask Bala to refresh the table.

Step1: At line 11 change the date to D-1<br><br>
Step2: At line around 36 (and (TSA_Opening_date < date '2023-10-18')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step3: At line around 45 (WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '2023-10-18')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step4: At line 58 (and TSA_Opening_date < date '2023-06-18'  and (Last_TSA_Credit_Date < date '2023-06-18' and Last_TSA_Debit_Date < date '2023-06-18' and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '2023-06-18')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) <br><br>
Step5: At line 72 (Run_Date = date '2024-01-09'###DATE change here to D-1) Change the date to D-1<br><br>

First Notification we change all the steps <br><br>
Second and Third Notification we only change Step1 and Step5
For Second Notificaiton and Third Notificaiton the customre id should be from the file sent by Marvin. <br><br>

In [19]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '{d_minus_1_days_string}' ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in {a}
)
select * from w_tsa;
"""

dfd = client.query(sq).to_dataframe()
dfd.shape

(8604, 29)

In [20]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '{d_minus_1_days_string}'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in {a}
),
w_approved
as
(
  select DISTINCT customerId from risk_credit_mis.loan_master_table where loantype = 'BIGLOAN' OR FLAGAPPROVAL = 1 OR FLAGDISBURSEMENT = 1  
  or (stage1Status = 'EXPIRED' AND cddInitiatedDateAndTime IS NULL)
)
select *
from
(
  select distinct
  a.Customer_Id,a.TSA_Opening_date,
  c.OFSTANDARDACCOUNTID as TSA_Account_Number, c.OfblockAmount, 
  a.TSA_Status,a.TSA_Balance,a.Solo_Stash_Balance,a.Group_Stash_Balance,a.Term_Deposit_Balance,a.Active_Term_Deposit_flag,a.Last_TSA_Credit_Date,a.Last_TSA_Debit_Date,
  a.ARI_Flag,a.Active_Loan,a.Total_Loan_Outstanding,  case when b.customerId is not null then 1 else 0 end as Approved_Loans,
  a.Pending_Loan_Applications,a.Hard_Reject_Loans_App,a.Soft_Reject_Loans_App,a.Cancelled_Loans_App, extract(date from a.Last_Login_Date) as Last_Login_Date,
  a.Last_TSA_EOD_Balance_Date,a.Last_Group_Stash_Credit_Date,a.Group_Stash_Balance_Owner,a.Run_Date,a.Batch,a.Processing_Month,a.pending_loans_not_expired_not_exempt,a.expired_loans,
  a.pending_loans_lastDt,a.expired_loans_lastDt,
  CASE 
    WHEN tsa_status = 'Active' and (Last_TSA_Credit_Date = date '1970-01-01' and Last_Group_Stash_Credit_Date is null)
    and (TSA_Opening_date < date '{d_minus_60_days_string}')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)  THEN 'TYPE 1'
    and b.customerId is null  THEN 'TYPE 1'


    WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '{d_minus_60_days_string}')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    and (date_trunc(TSA_Opening_date,day) = date_trunc(Last_Login_Date,day)) THEN 'TYPE 2'

    
    WHEN ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) < 1 )
            and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
            and (Active_Term_Deposit_flag = 0)
            and TSA_Opening_date < date '{d_minus_180_days_string}'                                     ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and (Last_TSA_Credit_Date < date '{d_minus_180_days_string}'                                 ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and Last_TSA_Debit_Date < date '{d_minus_180_days_string}'                                         ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '{d_minus_180_days_string}' )  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and tsa_status = 'Active'
            and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
            and b.customerId is null  
    THEN 'TYPE 3'
    ELSE 'NO ACTION'
    END AS ACTION_TYPE
  from w_tsa A
  left join w_approved b
  on cast(a.Customer_Id as int64) = b.customerId
  left join core_raw.customer_accounts c
  on c.OFCUSTOMERID = a.Customer_Id and c.CRINTERDESC = 'Transactional Savings Account Inv_R' and c.OFISCLOSED = 'N'
  where
  Run_Date = date '{d_minus_1_days_string}'  ###DATE change here to D-1
)
"""


In [21]:
print(sq)


with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '2024-05-23'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in ('2390807', '2407245', '2380295', '2392393', '2396066', '2386697', '2404268', '2378985', '2392962', '2392321', '2395479', '2403368', '2396301', '2385670', '2399473', '2273556', '2401601', '2398007', '2381327', '1289602', '2371970', '2406846', '2385440', '2390354', '2398175', '2374351', '2406019', '1138774', '2407972', '2395845', '2395977', '2387769', '2339326', '2382453', '2387073', '2378580', '2391293', '2243646',

In [22]:
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID e6ba26be-5115-475e-a629-99fb65698fed successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [23]:
pd.set_option('display.max_columns', None)
df.sample(10)

,Customer_Id,TSA_Opening_date,TSA_Account_Number,OfblockAmount,TSA_Status,TSA_Balance,Solo_Stash_Balance,Group_Stash_Balance,Term_Deposit_Balance,Active_Term_Deposit_flag,Last_TSA_Credit_Date,Last_TSA_Debit_Date,ARI_Flag,Active_Loan,Total_Loan_Outstanding,Approved_Loans,Pending_Loan_Applications,Hard_Reject_Loans_App,Soft_Reject_Loans_App,Cancelled_Loans_App,Last_Login_Date,Last_TSA_EOD_Balance_Date,Last_Group_Stash_Credit_Date,Group_Stash_Balance_Owner,Run_Date,Batch,Processing_Month,pending_loans_not_expired_not_exempt,expired_loans,pending_loans_lastDt,expired_loans_lastDt,ACTION_TYPE
6143,2399273,2024-02-13,60823992730001,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-02-14,NaT,NaT,NaN,2024-05-23,Batch_1,May,0,1,NaT,2024-05-14,TYPE 1
853,2398280,2024-02-12,60823982800005,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-05-03,NaT,NaT,NaN,2024-05-23,Batch_1,May,0,1,NaT,2024-05-15,TYPE 1
2265,2393255,2024-02-08,60823932550004,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-02-12,NaT,NaT,NaN,2024-05-23,Batch_1,May,0,1,NaT,2024-05-08,TYPE 1
1816,2406937,2024-02-21,60824069370006,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-02-21,NaT,NaT,NaN,2024-05-23,Batch_1,May,0,0,NaT,NaT,TYPE 1
6232,2394747,2024-02-09,60823947470003,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-02-09,NaT,NaT,NaN,2024-05-23,Batch_1,May,0,0,NaT,NaT,TYPE 1
2956,2406438,2024-02-20,60824064380002,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-03-04,NaT,NaT,NaN,2024-05-23,Batch_1,May,0,1,NaT,2024-05-21,TYPE 1
1561,2401422,2024-02-16,60824014220004,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,1,0,0,2024-02-17,NaT,NaT,NaN,2024-05-23,Batch_1,May,0,0,NaT,NaT,TYPE 1
8546,2383150,2024-01-29,60823831500001,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-01-29,NaT,NaT,NaN,2024-05-23,Batch_1,May,0,1,NaT,2024-04-29,TYPE 1
854,2329364,2024-01-26,60823293640007,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,1,0,2024-04-13,NaT,NaT,NaN,2024-05-23,Batch_1,May,0,1,NaT,2024-03-10,TYPE 1
585,2403732,2024-02-18,60824037320003,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-03-27,NaT,NaT,NaN,2024-05-23,Batch_1,May,0,0,NaT,NaT,TYPE 1


In [24]:
df.shape

(8604, 32)

In [25]:
df['ACTION_TYPE'].value_counts()

ACTION_TYPE
TYPE 1       7481
TYPE 3       1017
NO ACTION      86
TYPE 2         20
Name: count, dtype: int64

In [27]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\March24&April24 Ongoing Batch\20240524_TSA_Account_Closure_3rdNotification.xlsx", sheet_name='TSA_Closure_3rdNotification', index = False)